In [1]:
import cv2
import numpy as np
from PIL import Image
from IPython import display
import PIL
import random


In [2]:
car_pic = ""

pts = []

# Mouse-click event function credits: https://stackoverflow.com/questions/32770654/get-pixel-location-using-mouse-click-events/60445099#60445099

# click_event: stores a list of coordinate points clicked on by user 
# Args: 
#       event: type of click/event performed
#       x/y: coordinate positions
#       flags, param: any relevant flags or extra parameters provided by opencv library

def click_event(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONDOWN:
        print(x,",",y)
        # np.append(pts, [x,y])
        global pts
        pts.append([x,y])
        font = cv2.FONT_HERSHEY_SIMPLEX
        strXY = str(x)+", "+str(y)
        img = cv2.imread(car_pic)
        cv2.putText(img, strXY, (x,y), font, 0.5, (255,255,0), 2)
        cv2.imshow(car_pic, img)
        # print("PTS", pts)
        # return pts
    if event == cv2.EVENT_RBUTTONDOWN:
        return
        # return pts



# random_sticker: chooses random picture from ridesharing stickers collection
# args: none
# returns a random image
import glob 

def random_sticker():
    file_path = ["/home/tiwari/Waggle-Private Folder/Ridesharing Stickers/*.png", "/home/tiwari/Waggle-Private Folder/Ridesharing Stickers/*.jpg"]
    images = glob.glob(random.choice(file_path))
    random_img = random.choice(images)
    return random_img



# setSticker: overlays random sticker on chosen point (chosen car) in image
# Args: 
#       img_name: image path

def setSticker(img_name):
    
    global car_pic
    car_pic = img_name
    img = cv2.imread(car_pic)
    cv2.imshow(car_pic, img)
    cv2.setMouseCallback(img_name, click_event)
    global pts
    # print(pts)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    pts = np.float32(pts)

    #edge case - no cars in pic
    if (pts.size == 0):
        cv2.destroyAllWindows()
        return []
    

    sticker = cv2.imread('uber-airport.png')
    rndm_sticker = random_sticker()
    sticker = cv2.imread(rndm_sticker)


    sticker = cv2.cvtColor(sticker,cv2.COLOR_BGR2RGB)
    background = cv2.imread(img_name)
    background = cv2.cvtColor(background,cv2.COLOR_BGR2RGB)
    height = sticker.shape[0]
    width = sticker.shape[1]

    rows,cols,ch = background.shape

    pts1 = np.float32([[0,0],[width,0],[0,height],[width,height]]) # cola coords 
    pts2 = pts # stadium tile coords
    pts2 = np.float32(pts2)
    # print("pts", pts)

    M = cv2.getPerspectiveTransform(np.float32(pts1),np.float32(pts2))    
    dst = cv2.warpPerspective(sticker,M,(cols,rows))

    overlay = cv2.add(background, dst)
    # Image.fromarray(overlay).save("final22.png")
    return overlay


In [ ]:
# Divide into quadrants and overlay sticker on all node pictures

import os
from os import listdir
import glob

# Divide all images from Waggle Node W026 (from May 31 - June1) into quadrants and overlay sticker on each picture


count = 0
# folder_dir = '/Users/anaghatiwari/Documents/CarPics/storage.sagecontinuum.org/api/v1/data/imagesampler-bottom/sage-imagesampler-bottom-0.3.0/000048b02d05a1c7'
for images in glob.glob('/home/tiwari/Waggle-Private Folder/storage.sagecontinuum.org/api/v1/data/imagesampler-bottom/sage-imagesampler-bottom-0.3.0/000048b02d05a1c7/*.jpg'):
    name = os.path.dirname(images)
    car = cv2.imread(images)

    # print("IMAGES: ", images)

    (h, w) = car.shape[:2]
    (center_x, center_y) = (w//2, h//2)

    top_left = car[0:center_y, 0:center_x]
    top_right = car[0:center_y, center_x:w]
    bot_left = car[center_y:h, 0:center_x]
    bot_right = car[center_y:h, center_x:w]
    global pts
        
    # Store quadrant images as new file, call SetSticker function to overlay sticker in each quadrant

    for i in range(4):

        if i == 0:
            cv2.imwrite(str(count)+"q1.jpeg", top_left)
            overlay = setSticker(str(count) + "q1.jpeg")
            if (len(overlay) > 0):
                Image.fromarray(overlay).save(str(count) + "q1.jpeg")

        elif i ==1:
            pts = []
            cv2.imwrite(str(count)+'q2.jpeg', top_right)
            overlay = setSticker(str(count) + "q2.jpeg")
            if (len(overlay) >0):
                Image.fromarray(overlay).save(str(count) + "q2.jpeg")

        elif i ==2:
            pts = []
            cv2.imwrite(str(count)+'q3.jpeg', bot_left)

            overlay = setSticker(str(count) + "q3.jpeg")
            if (len(overlay) > 0):
                Image.fromarray(overlay).save(str(count) + "q3.jpeg")

        else:
            pts = []
            cv2.imwrite(str(count)+'q4.jpeg', bot_right)
            overlay = setSticker(str(count) + "q4.jpeg")
            if (len(overlay) > 0):
                Image.fromarray(overlay).save(str(count) + "q4.jpeg")
    
    # increments count (used to create new file name for each quadrant) 
    count += 1

    # print("COUNT: ", count)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()
    # print("next iteration now")

# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [ ]:
# Divide image from Waggle Node W026 into quadrants and overlay sticker on picture - testing code for one picture

import os
from os import listdir
import glob

 
# get the path/directory
#for images in os.listdir(folder_dir):

count = 0
images = "storage.sagecontinuum.org/api/v1/data/imagesampler-bottom/sage-imagesampler-bottom-0.3.0/000048b02d05a1c7/1685566821898592132-sample.jpg"
car = cv2.imread(images)
(h, w) = car.shape[:2]
(center_x, center_y) = (w//2, h//2)

# Divide image into 4 quadrants
top_left = car[0:center_y, 0:center_x]
top_right = car[0:center_y, center_x:w]
bot_left = car[center_y:h, 0:center_x]
bot_right = car[center_y:h, center_x:w]
global pts

for i in range(4):
    # Store quadrant images as new file, call SetSticker function to overlay sticker in each quadrant
    if i == 0:
        cv2.imwrite(str(count)+"q1.jpeg", top_left)
        overlay = setSticker(str(count) + "q1.jpeg")
        if (len(overlay) > 0):
            Image.fromarray(overlay).save(str(count) + "q1.jpeg")

    elif i ==1:
        pts = []
        cv2.imwrite(str(count)+'q2.jpeg', top_right)
        overlay = setSticker(str(count) + "q2.jpeg")
        if (len(overlay) >0):
            Image.fromarray(overlay).save(str(count) + "q2.jpeg")

    elif i ==2:
        pts = []
        cv2.imwrite(str(count)+'q3.jpeg', bot_left)

        overlay = setSticker(str(count) + "q3.jpeg")
        if (len(overlay) > 0):
            Image.fromarray(overlay).save(str(count) + "q3.jpeg")

    else:
        pts = []
        cv2.imwrite(str(count)+'q4.jpeg', bot_right)
        overlay = setSticker(str(count) + "q4.jpeg")
        if (len(overlay) > 0):
            Image.fromarray(overlay).save(str(count) + "q4.jpeg")

count += 1
cv2.waitKey(0)
cv2.destroyAllWindows()


In [ ]:
cv2.destroyAllWindows()